In [ ]:
from cng.utils import set_secrets
import ibis
from ibis import _
from tqdm import tqdm


In [ ]:

import duckdb
con = duckdb.connect()
    

path = 's3://gbif-open-data-us-east-1/occurrence/2025-06-01/occurrence.parquet/**'

con.execute("""
CREATE SECRET IF NOT EXISTS gbif_public (
    TYPE S3,
    KEY_ID '',
    SECRET '',
    REGION 'us-east-1',
    SCOPE 's3://gbif-open-data-us-east-1'
);
""")
con.read_parquet(path)

IOException: IO Error: No files found that match the pattern "s3://gbif-open-data-us-east-1/occurrence/2025-06-01/occurrence.parquet/**/*.parquet"

In [ ]:
con = ibis.duckdb.connect("tmp.duckdb")
con.raw_sql("SET enable_progress_bar = false")
con.raw_sql("SET memory_limit = '50GB';")

set_secrets(con) 

gbif = con.read_parquet("s3://public-gbif/2024-10-01/**")

In [ ]:
lvl = "h0" 
hex_sets = gbif.group_by(_[lvl]).agg(n = _.count()).order_by(_.n.desc()).execute()[lvl]

In [ ]:
for hexid in tqdm(hex_sets):
    gbif.filter(_[lvl] == hexid).to_parquet(f"s3://public-gbif/hex/{lvl}={hexid}/part0.parquet")

In [ ]:
## resume with the missing hexes if necessary...
# done = con.read_parquet("s3://public-gbif/hexid/**").select(lvl).distinct()
# hex_sets = gbif.group_by(_[lvl]).agg(n = _.count()).anti_join(done, lvl).order_by(_.n.desc()).execute()[lvl]